In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import time

import tkinter as tk
from tkinter import ttk
import tkinter.messagebox as msg # messagebox要另行匯入，否則會出錯。
from tkcalendar import Calendar, DateEntry

import plotly.express as px

## 備註
1. 顯示的圖片先以企劃中截止日期最大的為主，也可以透過參數輸入想觀看的截止日
2. 參考: https://plotly.com/python/text-and-annotations/

In [2]:
## 輸入今日日期，作為甘特圖的起始日，也可以透過參數輸入想觀看的起始日
today = datetime.now().strftime('%Y-%m-%d')
today

'2023-01-29'

In [3]:
## 取得user name 
user_name = os.getlogin()
user_name

'BoDun'

## 甘特圖
#### 狀態分類: 完成、執行中、未開始
#### 規則:
    1. 擷取今日日期，若今日日期 >= 啟動時間，則調整為「執行中」。
    2. 做一個小工具，透過輸入參數的方式確認是否有要調整為完成。
    3. 若距離截止日期僅剩五天，則輸入剩餘天數

In [4]:
def get_gantt():
    
    ## 讀取資料時會先透過截止日期排序
    file = pd.read_excel('待辦.xlsx').sort_values('截止日期').reset_index(drop = True)

    ## 規則一
    filter1 = file['啟動時間'] <= today
    filter2 = file['狀態'] == '未開始'
    file.loc[filter1 & filter2, "狀態"] = '執行中'
    
    ## 計算剩餘天數
    file['剩餘天數'] = (file['截止日期'].apply(pd.to_datetime) - pd.to_datetime(today)).dt.days
    file['剩餘天數_2'] = list(map(lambda x : '剩餘天數: '+str(x)+' 天', file['剩餘天數']))
    
    ## 透過 tkinter篩選器 挑選任務狀態
    filter_ = []
    if gantt_cbtn_checkbutton1.get() == 1:
        filter_.append('完成')
    if gantt_cbtn_checkbutton2.get() == 1:
        filter_.append('執行中')
    if gantt_cbtn_checkbutton3.get() == 1:
        filter_.append('未開始')    
    
    file = file[file['狀態'].isin(filter_)].sort_values('截止日期').reset_index(drop = True)
        
    ## 新增 index 欄位
    file.insert(loc = 0, column = 'index', value = list(range(len(file))))
    
    ## 製作甘特圖
    fig = px.timeline(file, 
                      x_start="啟動時間", 
                      x_end="截止日期", 
                      y="工作內容", 
                      color = '狀態',
                      color_discrete_map = {"完成": "blue", "執行中": "black", "未開始": "green"},
                      #facet_row = '負責人',
                      text = '剩餘天數_2',
                      title = '待辦事項'
                     )
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(paper_bgcolor = "white",
                      plot_bgcolor = "white",
                      showlegend = True ## 是否顯示小圖示
                     )
    fig.add_vline(x=today, line_width=3, line_dash="dash", line_color="green")
    
    ## 剩餘天數為0~5天的任務標出醒目紅色提醒
    red_color_data = file[(file['剩餘天數'] <= int(gantt_en_reminder.get())) & (file['剩餘天數'] > 0)]
    if len(red_color_data) == 0:
        pass
    else:
        for i in red_color_data['index']:
            opacity_ = (6-red_color_data[red_color_data['index'] == i]['剩餘天數'].values[0])/10
            fig.add_hrect(y0 = (-0.5 + i), 
                          y1 = (0.5 + i),
                          line_width = 0, 
                          fillcolor = "red", 
                          opacity = opacity_)
    
    fig.write_image("img.png")
    
    image = tk.PhotoImage(file = "img.png")
    gantt_lb_imageLabel.configure(image = image)
    gantt_lb_imageLabel.image = image
    

## Tkinter
* 變數命名規則: 區塊名稱_物件類型代稱_簡單說明

* 物件類型代稱: 
> * label : lb
> * entry : en
> * button : btn
> * Radiobutton : rbtn
> * Checkbutton : cbtn

In [5]:
def choose_startdt():
    def print_sel():      
        if task_add_en_startdt.get() is None:
            task_add_en_startdt.insert(0, cal.selection_get()) 
            top.destroy() ## 關閉視窗
        else:
            task_add_en_startdt.delete(0, 'end')
            task_add_en_startdt.insert(0, cal.selection_get()) 
            top.destroy() ## 關閉視窗
            
    top = tk.Toplevel(window)
    top.iconbitmap('卡咪龜.ico')
    top.title('日期選擇')
    
    cal = Calendar(top,
                   font="Arial 14", 
                   selectmode='day',
                   cursor="hand1",
                   year = int(today.split('-')[0]), 
                   month = int(today.split('-')[1]), 
                   day = int(today.split('-')[2]))
    cal.pack(fill="both", expand=True)
    ttk.Button(top, text="ok", command=print_sel).pack()

In [6]:
def choose_enddt():
    def print_sel():      
        if task_add_en_enddt.get() is None:
            task_add_en_enddt.insert(0, cal.selection_get()) 
            top.destroy() ## 關閉視窗
        else:
            task_add_en_enddt.delete(0, 'end')
            task_add_en_enddt.insert(0, cal.selection_get()) 
            top.destroy() ## 關閉視窗
            
    top = tk.Toplevel(window)
    top.iconbitmap('卡咪龜.ico')
    top.title('日期選擇')
    
    cal = Calendar(top,
                   font="Arial 14", 
                   selectmode='day',
                   cursor="hand1",
                   year = int(today.split('-')[0]), 
                   month = int(today.split('-')[1]), 
                   day = int(today.split('-')[2]))
    cal.pack(fill="both", expand=True)
    ttk.Button(top, text="ok", command=print_sel).pack()

In [7]:
## 新增資料到 excel 中
def get_renew_data():
    new_data = []
    new_data.append(task_add_en_startdt.get())
    new_data.append(task_add_en_enddt.get())
    new_data.append(task_add_en_content.get())
    new_data.append(task_add_en_detail.get())
    new_data.append(task_add_en_manager.get())
    new_data.append(task_add_en_importance.get())
    new_data.append('未開始')

    ## 讀取原始excel
    file = pd.read_excel('待辦.xlsx').sort_values('截止日期').reset_index(drop = True)
    file.loc[file.index.stop] = new_data
    file.to_excel('待辦.xlsx', index = False)

In [8]:
def get_combobox_list():
    
    ## 讀取資料時會先透過截止日期排序
    file = pd.read_excel('待辦.xlsx').sort_values('截止日期').reset_index(drop = True)
    
    ## 規則一
    filter1 = file['啟動時間'] <= today
    filter2 = file['狀態'] == '未開始'
    file.loc[filter1 & filter2, "狀態"] = '執行中'
    
    ## 透過 tkinter篩選器 挑選任務狀態
    filter_ = []
    if task_detail_cbtn_checkbutton1.get() == 1:
        filter_.append('完成')
    if task_detail_cbtn_checkbutton2.get() == 1:
        filter_.append('執行中')
    if task_detail_cbtn_checkbutton3.get() == 1:
        filter_.append('未開始')    
    
    file = file[file['狀態'].isin(filter_)].sort_values('截止日期').reset_index(drop = True)
    
    task_detail_combo_tasklist['value'] = list(file['工作內容'])

In [9]:
def get_combobox_selected_detail(event):
    
    file = pd.read_excel('待辦.xlsx').sort_values('截止日期').reset_index(drop = True)
    text = file[file['工作內容'] == comboboxText.get()]['工作細項'].values[0]
    
    task_detail_lb_detail_text.set(text)

In [14]:
################################################ 基本設定 ################################################
window = tk.Tk()
window.title('TODO_TOOL')
window.geometry('1535x790+-10+0') ##('380x400') # 修改視窗大小(寬x高); +x+y是視窗距離螢幕的距離
window.resizable(False, False) # 如果不想讓使用者能調整視窗大小的話就均設為False 
window.iconbitmap('卡比獸.ico') # 更改左上角的icon圖示

################################################ 標題區塊 ################################################
title_fm = tk.Frame(window, bg = 'white', height = 70) # bg='Royal Blue'
title_fm.pack(fill = tk.BOTH)

title_lb_1 = tk.Label(title_fm, text = "TO DO ", font = ("Microsoft JhengHei", 20), bg = "white")
title_lb_1.place(x = 767, y = 0)

title_lb_2 = tk.Label(title_fm, text = "Never put off till tomorrow what you can do today.", font = ("Microsoft JhengHei", 10), bg = "white")
title_lb_2.place(x = 667, y = 40)

title_lb_date = tk.Label(title_fm, text = "今日日期: "+today, font = ("Microsoft JhengHei", 10), bg = "white")
title_lb_date.place(x = 1350, y = 10)

title_lb_author = tk.Label(title_fm, text = "作者: "+user_name, font = ("Microsoft JhengHei", 10), bg = "white")
title_lb_author.place(x = 1350, y = 40)

################################################ 新增任務及調整區塊 ################################################
task_add_fm = tk.Frame(window, bg = 'white', height = 120) # bg = 'Gold'
task_add_fm.pack(fill = tk.BOTH)

## Label 區域
task_add_lb_title = tk.Label(task_add_fm, text = "任務新增", font = ("Microsoft JhengHei", 15), bg = "white")
task_add_lb_title.place(x = 0, y = 0)

task_add_lb_startdt = tk.Label(task_add_fm, text = "啟動日期", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_startdt.place(x = 100, y = 30)

task_add_lb_enddt = tk.Label(task_add_fm, text = "截止日期", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_enddt.place(x = 255, y = 30)

task_add_lb_content = tk.Label(task_add_fm, text = "工作內容", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_content.place(x = 410, y = 30)

task_add_lb_detail = tk.Label(task_add_fm, text = "工作細項", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_detail.place(x = 585, y = 30)

task_add_lb_manager = tk.Label(task_add_fm, text = "負責人", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_manager.place(x = 800, y = 30)

task_add_lb_importance = tk.Label(task_add_fm, text = "重要程度", font = ("Microsoft JhengHei", 12), bg = "white")
task_add_lb_importance.place(x = 900, y = 30)

## Button 區域
task_add_btn_startdt = tk.Button(task_add_fm, text = "...", command = choose_startdt)
task_add_btn_startdt.place(x = 226, y = 60)

task_add_btn_enddt = tk.Button(task_add_fm, text = "...", command = choose_enddt)
task_add_btn_enddt.place(x = 381, y = 60)

task_add_btn_renew = tk.Button(task_add_fm, text = "更新", font = ("Microsoft JhengHei", 10), command = get_renew_data)
task_add_btn_renew.place(x = 950, y = 85)

## Entry 區域
task_add_en_startdt = tk.Entry(task_add_fm, width = 15, font = ("Microsoft JhengHei", 10))
task_add_en_startdt.place(x = 100, y = 60)

task_add_en_enddt = tk.Entry(task_add_fm, width = 15, font = ("Microsoft JhengHei", 10))
task_add_en_enddt.place(x = 255, y = 60)

task_add_en_content = tk.Entry(task_add_fm, width = 20, font = ("Microsoft JhengHei", 10))
task_add_en_content.place(x = 410, y = 60)

task_add_en_detail = tk.Entry(task_add_fm, width = 25, font = ("Microsoft JhengHei", 10))
task_add_en_detail.place(x = 585, y = 60)

task_add_en_manager = tk.Entry(task_add_fm, width = 10, font = ("Microsoft JhengHei", 10))
task_add_en_manager.insert(0, user_name)
task_add_en_manager.place(x = 800, y = 60)

task_add_en_importance = tk.Entry(task_add_fm, width = 10, font = ("Microsoft JhengHei", 10))
task_add_en_importance.insert(0, 1)
task_add_en_importance.place(x = 900, y = 60)

################################################ 甘特圖 ################################################
gantt_fm = tk.Frame(window, bg = 'white', height = 480) # bg = 'Aquamarine'
gantt_fm.pack(fill = tk.BOTH)

## Label 區域
gantt_lb_title = tk.Label(gantt_fm, text = "甘特圖", font = ("Microsoft JhengHei", 15), bg = "white")
gantt_lb_title.place(x = 0, y = 0)

gantt_lb_illustrate1 = tk.Label(gantt_fm, text = "請勾選任務狀態(可複選)", font = ("Microsoft JhengHei", 12), bg = "white")
gantt_lb_illustrate1.place(x = 10, y = 40)

gantt_lb_illustrate2 = tk.Label(gantt_fm, text = "欲醒目提示的剩餘天數", font = ("Microsoft JhengHei", 12), bg = "white")
gantt_lb_illustrate2.place(x = 10, y = 100)

gantt_lb_illustrate3 = tk.Label(gantt_fm, text = "當執行天數剩餘: ", font = ("Microsoft JhengHei", 10), bg = "white")
gantt_lb_illustrate3.place(x = 20, y = 130)

gantt_lb_illustrate4 = tk.Label(gantt_fm, text = " 天時，以紅色醒目提示", font = ("Microsoft JhengHei", 10), bg = "white")
gantt_lb_illustrate4.place(x = 155, y = 130)

gantt_lb_illustrate5 = tk.Label(gantt_fm, text = "備註", font = ("Microsoft JhengHei", 12), bg = "white")
gantt_lb_illustrate5.place(x = 10, y = 160)

gantt_lb_illustrate6 = tk.Label(gantt_fm, text = "1. 圖中綠色虛線表今日日期", font = ("Microsoft JhengHei", 10), bg = "white")
gantt_lb_illustrate6.place(x = 20, y = 190)

gantt_lb_imageLabel = tk.Label(gantt_fm)
gantt_lb_imageLabel.place(x = 467, y = 0)

## Button 區域
gantt_btn_renew = tk.Button(gantt_fm, text = "更新", font = ("Microsoft JhengHei", 10), command = get_gantt)
gantt_btn_renew.place(x = 250, y = 220)

## Entry 區域
gantt_en_reminder = tk.Entry(gantt_fm, width = 3, font = ("Microsoft JhengHei", 10))
gantt_en_reminder.insert(0, 3)
gantt_en_reminder.place(x = 125, y = 130)

## Checkbutton 區域
gantt_cbtn_checkbutton1 = tk.IntVar()
gantt_cbtn_checkbutton1_1 = tk.Checkbutton(gantt_fm, 
                                           text = '完成', 
                                           variable = gantt_cbtn_checkbutton1, font = ("Microsoft JhengHei", 10),
                                           bg = "white")
gantt_cbtn_checkbutton1_1.place(x = 20, y = 70)

gantt_cbtn_checkbutton2 = tk.IntVar()
gantt_cbtn_checkbutton2_1 = tk.Checkbutton(gantt_fm, 
                                           text = '執行中', 
                                           variable = gantt_cbtn_checkbutton2, font = ("Microsoft JhengHei", 10),
                                           bg = "white")
gantt_cbtn_checkbutton2_1.place(x = 80, y = 70)
gantt_cbtn_checkbutton2_1.select() ## 預設勾選

gantt_cbtn_checkbutton3 = tk.IntVar()
gantt_cbtn_checkbutton3_1 = tk.Checkbutton(gantt_fm, 
                                           text = '未開始', 
                                           variable = gantt_cbtn_checkbutton3, font = ("Microsoft JhengHei", 10),
                                           bg = "white")
gantt_cbtn_checkbutton3_1.place(x = 130, y = 70)
gantt_cbtn_checkbutton3_1.select() ## 預設勾選

################################################ 任務細項說明區 ################################################
task_detail_fm = tk.Frame(window, bg = 'white', height = 118) # bg = 'Violet'
task_detail_fm.pack(fill = tk.BOTH)

## Label 區域
task_detail_lb_title = tk.Label(task_detail_fm, text = "任務細項說明", font = ("Microsoft JhengHei", 15), bg = "white")
task_detail_lb_title.place(x = 0, y = 0)

task_detail_lb_illustrate1 = tk.Label(task_detail_fm, text = "請勾選任務狀態(可複選)", font = ("Microsoft JhengHei", 10), bg = "white")
task_detail_lb_illustrate1.place(x = 10, y = 30)

task_detail_lb_illustrate2 = tk.Label(task_detail_fm, text = "請選擇欲顯示細項的任務", font = ("Microsoft JhengHei", 10), bg = "white")
task_detail_lb_illustrate2.place(x = 250, y = 30)

task_detail_lb_illustrate3 = tk.Label(task_detail_fm, text = "細項說明", font = ("Microsoft JhengHei", 10), bg = "white")
task_detail_lb_illustrate3.place(x = 450, y = 30)

task_detail_lb_detail_text = tk.StringVar()
task_detail_lb_detail_text1 = tk.Label(task_detail_fm, textvariable = task_detail_lb_detail_text, font=('Microsoft JhengHei', 8))
task_detail_lb_detail_text1.place(x = 450, y = 55)

## Button 區域
task_detail_btn_check_state = tk.Button(task_detail_fm, text = "篩選完成", font = ("Microsoft JhengHei", 8), command = get_combobox_list)
task_detail_btn_check_state.place(x = 157, y = 85)

## Checkbutton 區域
task_detail_cbtn_checkbutton1 = tk.IntVar()
task_detail_cbtn_checkbutton1_1 = tk.Checkbutton(task_detail_fm, 
                                           text = '完成', 
                                           variable = task_detail_cbtn_checkbutton1, font = ("Microsoft JhengHei", 8),
                                           bg = "white")
task_detail_cbtn_checkbutton1_1.place(x = 20, y = 55)

task_detail_cbtn_checkbutton2 = tk.IntVar()
task_detail_cbtn_checkbutton2_1 = tk.Checkbutton(task_detail_fm, 
                                           text = '執行中', 
                                           variable = task_detail_cbtn_checkbutton2, font = ("Microsoft JhengHei", 8),
                                           bg = "white")
task_detail_cbtn_checkbutton2_1.place(x = 80, y = 55)

task_detail_cbtn_checkbutton3 = tk.IntVar()
task_detail_cbtn_checkbutton3_1 = tk.Checkbutton(task_detail_fm, 
                                           text = '未開始', 
                                           variable = task_detail_cbtn_checkbutton3, font = ("Microsoft JhengHei", 8),
                                           bg = "white")
task_detail_cbtn_checkbutton3_1.place(x = 150, y = 55)

## Combobox 區域
comboboxText = tk.StringVar()
task_detail_combo_tasklist = ttk.Combobox(task_detail_fm, textvariable = comboboxText)
task_detail_combo_tasklist.place(x = 250, y = 55)

task_detail_combo_tasklist.bind('<<ComboboxSelected>>', get_combobox_selected_detail)

################################################ 啟動!!!!! ################################################
window.mainloop()